In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt

# Подключаем проект
PROJECT_ROOT = os.path.abspath("..")
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

from src.models.scoring import add_uplift_scores
from src.evaluation.uplift import compute_qini, compute_auuc, plot_uplift_curve

In [ ]:
NBO_DATASET_PATH = os.path.join(PROJECT_ROOT, "data", "processed", "nbo_dataset.csv")

df = pd.read_csv(NBO_DATASET_PATH)
df.head()

In [ ]:
df_uplift = add_uplift_scores(df)

df_uplift[["p_treat", "p_control", "uplift"]].head()

In [ ]:
df_uplift["uplift"].describe()

In [ ]:
qini = compute_qini(df_uplift)
print(f"Qini coefficient: {qini:.4f}")

In [ ]:
auuc = compute_auuc(df_uplift)
print(f"AUUC: {auuc:.4f}")

In [ ]:
plot_uplift_curve(df_uplift)

In [ ]:
def uplift_at_k(df, k=0.3):
    n = int(len(df) * k)
    top_df = df.sort_values("uplift", ascending=False).head(n)

    treat_gain = (top_df[top_df["treatment"] == 1]["outcome_click"].sum())
    ctrl_gain = (top_df[top_df["treatment"] == 0]["outcome_click"].sum())

    return treat_gain - ctrl_gain

for k in [0.05, 0.1, 0.2, 0.3, 0.5]:
    score = uplift_at_k(df_uplift, k)
    print(f"uplift@{k}: {score:.4f}")

In [ ]:
summary = pd.DataFrame({
    "metric": ["Qini", "AUUC"],
    "value": [qini, auuc]
})

summary